In [1]:
# === SYSTEM & IMPORTS ===
# Dieser Block importiert alle notwendigen Bibliotheken und prüft die Systemumgebung.

# os: Betriebssystem-Funktionen (Pfade erstellen, Umgebungsvariablen lesen)
# sys: System-Parameter (z.B. Python-Pfad)
# json: Konfigurationsdateien lesen/schreiben
# time: Zeitstempel für Ordnernamen und Messungen
# glob: Dateimuster-Suche (z.B. "*.csv" finden)
import os, sys, json, time, glob

# Path: Objektorientierter Umgang mit Dateipfaden
from pathlib import Path

# numpy (np): Schnelle numerische Operationen auf Arrays
import numpy as np

# pandas (pd): Tabellarische Datenverarbeitung (DataFrames)
import pandas as pd

# Wir setzen das Root-Verzeichnis auf den übergeordneten Ordner (..), 
# damit Python unsere eigenen Module (z.B. im Parent-Ordner) findet.
ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

In [2]:
# === 0) KONFIGURATION LADEN ===
# Wir laden die zentrale Steuerdatei 'config.json'. Das stellt sicher, dass alle Skripte 
# mit denselben Parametern (Ticker, Zeiträume etc.) arbeiten.

with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

# Wir extrahieren die Parameter in eigene Variablen für bessere Lesbarkeit.
TICKER   = C["ticker"]       # Börsenkürzel (z.B. AAPL)
START    = C["start"]        # Startdatum der Daten
END      = C["end"]          # Enddatum der Daten
INTERVAL = C["interval"]     # Daten-Intervall (z.B. 1d)

# Modell-Parameter aus der Config:
HORIZON  = int(C["horizon"])  # Vorhersage-Horizont: Für wie viele Tage in die Zukunft sagen wir vorher?
LOOKBACK = int(C["lookback"]) # Rückblick: Wie viele vergangene Tage sieht das Modell als Input?
BATCH    = int(C["batch"])    # Batch-Größe: Anzahl der Samples pro Trainings-Schritt
EPOCHS   = int(C["epochs"])   # Epochen: Wie oft wird der gesamte Datensatz trainiert?
SEED     = int(C.get("seed", 42)) # Seed für Reproduzierbarkeit

# Feature- und Label-Einstellungen:
FEATURESET = C.get("featureset", "v2") # Welches Variablenset nutzen wir?
EPS_MODE   = C.get("epsilon_mode", "abs") # Wie wird "Up" definiert? (Absolut oder Quantil)
EPSILON    = float(C.get("epsilon", 0.0005)) # Der Schwellwert für "Up"

# Ergebnis-Ordner vorbereiten.
# Jedes Training bekommt einen eigenen Unterordner mit Zeitstempel, damit nichts überschrieben wird.
RESULTS_DIR = Path(C.get("results_dir", "../results"))
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Generiere Zeitstempel: YYYY-MM-DD_HH-MM-SS_lstm
run_name = time.strftime("%Y-%m-%d_%H-%M-%S_lstm")
RUN_DIR  = RESULTS_DIR / run_name
RUN_DIR.mkdir(parents=True, exist_ok=True)

print("RUN_DIR:", RUN_DIR)

RUN_DIR: ..\results\2026-01-03_21-02-35_lstm


In [3]:
# === ABLATIONS STUDIEN (EXPERIMENTELLE SCHALTER) ===
# "Ablation Studies" bedeuten, dass man gezielt Teile des Modells weglässt oder ändert,
# um zu verstehen, welchen Beitrag sie zur Leistung liefern.

AB = C.get("ablations", {}) # Lädt Ablation-Settings aus der Config (falls vorhanden)

# Hilfsfunktion, um Boolesche Werte sicher zu lesen (auch aus Umgebungsvariablen).
def _get_bool(key, default):
    env = os.getenv(key)
    if env is not None:
        # Prüft auf verschiedene Schreibweisen für "Wahr"
        return env.strip().lower() in ("1","true","yes","y","on")
    return bool(AB.get(key.lower(), default))

# 1. Shuffle Train: Sollen die Trainingsdaten zufällig gemischt werden?
# Standard: True (wichtig, damit das Modell nicht die Reihenfolge auswendig lernt)
ABL_SHUFFLE_TRAIN = _get_bool("ABLATION_SHUFFLE_TRAIN", True)

# 2. Recurrent Dropout: Soll Dropout innerhalb der LSTM-Zellen deaktiviert werden?
# Dropout hilft gegen Overfitting, macht das Training aber auf GPUs oft langsamer.
# Standard: False (also Dropout benutzen)
ABL_NO_RECURRENT_DROPOUT = _get_bool("ABLATION_NO_RECURRENT_DROPOUT", False)

# 3. LayerNormalization Layout: Wo sollen die Normalisierungs-Schichten hin?
# "both" = Nach jeder RNN-Schicht.
# "after_second" = Nur nach der zweiten Schicht.
ABL_LN_LAYOUT = os.getenv("ABLATION_LN_LAYOUT", AB.get("ln_layout", "both")).lower()
if ABL_LN_LAYOUT not in {"both","after_second"}:
    ABL_LN_LAYOUT = "both"

print(f"[Ablations] shuffle_train={ABL_SHUFFLE_TRAIN} | no_recurrent_dropout={ABL_NO_RECURRENT_DROPOUT} | ln_layout={ABL_LN_LAYOUT}")

# Wir konstruieren den erwarteten Dateinamen für die Trainingsdaten.
# Dies dient nur zur Info, geladen wird später dynamischer.
eps_tag   = f"{EPS_MODE}{str(EPSILON).replace('.','p')}"
TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}_{eps_tag}.csv"

# Import von TensorFlow und Keras für das Deep Learning
import tensorflow as tf
from tensorflow import keras
# Metriken zur Bewertung der Klassifikation
from sklearn.metrics import (
    classification_report, confusion_matrix,
    balanced_accuracy_score, matthews_corrcoef, average_precision_score,
    roc_auc_score
)

[Ablations] shuffle_train=True | no_recurrent_dropout=False | ln_layout=both


In [4]:
# === 1) BESTE CONFIG LADEN (Aus Hyperparameter-Optimierung) ===
# Falls wir vorher einen Optimierungslauf (WFCV) gemacht haben, wollen wir dessen
# Ergebnisse (die besten Parameter) hier nutzen.

def _latest_best_config(results_dir="../results"):
    # Wir suchen nach allen 'best_config.json' Dateien in WFCV-Ordnern
    pattern = os.path.join(results_dir, "*_wfcv", "best_config.json")
    cands = glob.glob(pattern)
    if not cands:
        return None, None
    # Wir sortieren nach Änderungsdatum, um die neueste zu finden
    cands = sorted(cands, key=os.path.getmtime)
    best_path = cands[-1]
    with open(best_path, "r") as f:
        best_cfg = json.load(f)
    return best_cfg, best_path

# Aufruf der Suchfunktion
BEST_CFG, BEST_CFG_PATH = _latest_best_config(RESULTS_DIR)

# Fallback: Wenn keine Optimierung lief, nehmen wir Standardwerte.
if BEST_CFG is None:
    print("[INFO] Keine best_config.json gefunden — nutze Fallback (Config.json-Defaults).")
    BEST_CFG = {
        "features_used": "all",
        "lookback": LOOKBACK,
        "cell": "GRU",
        "width1": 32,
        "width2": 16,
        "dropout": 0.10,
        "lr": 5e-4
    }
else:
    print("Gefunden best_config.json:", BEST_CFG_PATH)

# Wir setzen die Hyperparameter basierend auf der geladenen Config (oder Fallback).
CELL    = str(BEST_CFG.get("cell", "GRU")).upper() # Typ der RNN-Zelle: GRU oder LSTM
WIDTH1  = int(BEST_CFG.get("width1", 32))          # Anzahl Neuronen in 1. Schicht
WIDTH2  = int(BEST_CFG.get("width2", 16))          # Anzahl Neuronen in 2. Schicht
DROPOUT = float(BEST_CFG.get("dropout", 0.10))     # Dropout-Rate (z.B. 0.10 = 10% Neuronen abschalten)
LR      = float(BEST_CFG.get("lr", 5e-4))          # Lernrate für den Optimizer
LB_FROM_BEST = int(BEST_CFG.get("lookback", LOOKBACK)) # Wichtiger Parameter: Lookback

# Wir nutzen den Lookback aus der Optimierung, falls vorhanden, sonst den aus der config.json
USE_LOOKBACK = LB_FROM_BEST if LB_FROM_BEST > 0 else LOOKBACK
FEATURES_USED_TAG = str(BEST_CFG.get("features_used", "all"))

# Ablation-Logik anwenden:
# Wenn "no_recurrent_dropout" aktiv ist, setzen wir Dropout in den RNN-Zellen auf 0.
# Dafür erhöhen wir ggf. die L2-Regularisierung im Dense-Layer, um Overfitting anders zu bekämpfen.
if ABL_NO_RECURRENT_DROPOUT:
    RDROP = 0.0
    L2_DENSE = 1e-4 
else:
    RDROP = DROPOUT
    L2_DENSE = 1e-5

print(f"[Block3 Setup] cell={CELL} width={WIDTH1}/{WIDTH2} rd={RDROP} dp_cfg={DROPOUT} lr={LR} lookback={USE_LOOKBACK} | features_used={FEATURES_USED_TAG} | L2(Dense)={L2_DENSE}")

Gefunden best_config.json: ..\results\2026-01-03_20-56-59_wfcv\best_config.json
[Block3 Setup] cell=GRU width=64/32 rd=0.2 dp_cfg=0.2 lr=0.0005 lookback=60 | features_used=all | L2(Dense)=1e-05


In [5]:
# === 2) DATEN & FEATURES LADEN ===
# Hier laden wir die vorbereiteten Trainingsdaten (CSV).

import yaml, glob, os, re

# Wir laden die Dokumentation unserer Features, um zu wissen, welche Spalten existieren.
yaml_path = f"../data/features_{FEATURESET}.yml"
meta = {}
if os.path.exists(yaml_path):
    with open(yaml_path, "r") as f:
        meta = yaml.safe_load(f) or {}

# Funktion, um die korrekte CSV-Datei zu finden.
# Da der Dateiname viele Parameter enthält (Epsilon, Horizon etc.), müssen wir flexibel suchen.
def _resolve_train_csv():
    # 1. Versuch: Exakter Pfad basierend auf Konfiguration
    eps_tag_cfg = f"{EPS_MODE}{str(EPSILON).replace('.','p')}"
    exact = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}_{eps_tag_cfg}.csv"
    if os.path.exists(exact):
        return exact
        
    # 2. Versuch: Pfad basierend auf den Einträgen im YAML (falls dort abweichend definiert)
    lab = (meta or {}).get("label", {})
    h_yaml   = int(lab.get("horizon", HORIZON))
    mode_yaml= str(lab.get("mode", EPS_MODE))
    eps_yaml = float(lab.get("epsilon", EPSILON))
    eps_tag_yaml = f"{mode_yaml}{str(eps_yaml).replace('.','p')}"
    by_yaml = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{h_yaml}_{eps_tag_yaml}.csv"
    if os.path.exists(by_yaml):
        return by_yaml
        
    # 3. Versuch: Irgendeine passende Datei finden (Fallback)
    pat_any = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h*.csv"
    cands = sorted(glob.glob(pat_any), key=os.path.getmtime)
    if cands:
        return cands[-1] # Die neueste nehmen
        
    raise FileNotFoundError("Kein TRAIN_CSV gefunden. Bitte Block 2 mit Label-Definition laufen lassen.")

# Datei bestimmen und Pfad ausgeben
TRAIN_CSV = _resolve_train_csv()
print("Loaded TRAIN_CSV:", TRAIN_CSV)

# Wir analysieren den Dateinamen, um sicherzugehen, welche Label-Parameter wirklich drinstecken.
def _infer_label_from(meta_dict, train_csv_path, fallback_h_from_root):
    h, mode, eps = None, None, None
    # Regex sucht nach Muster wie "_cls_h5_abs0p0005.csv"
    m = re.search(r"_cls_h(\d+)_([a-zq]+)([0-9p.]+)\.csv$", str(train_csv_path))
    if m:
        h    = int(m.group(1))
        mode = m.group(2)
        eps_str = m.group(3)
        eps = float(str(eps_str).replace("p", "."))
    
    if h is None: h = int(fallback_h_from_root)
    return h, mode, eps

H_DATA, MODE_DATA, EPS_DATA = _infer_label_from(meta, TRAIN_CSV, HORIZON)
HORIZON  = int(H_DATA)
if MODE_DATA is not None:   EPS_MODE = str(MODE_DATA)
if EPS_DATA  is not None:   EPSILON  = float(EPS_DATA)
print(f"[Label] using horizon={HORIZON} | mode={EPS_MODE} | epsilon={EPSILON}")

# Daten einlesen. Index ist das Datum.
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()

# Feature-Auswahl: Welche Spalten nutzen wir als Input (X)?
# Wir holen uns alle Features aus der YAML-Liste, die auch im DataFrame sind.
ALL_FEATURES = [c for c in (meta.get("features", []) if meta else []) if c in df.columns]

# Fallback, falls YAML leer: Alles außer OHLCV (Preise) und Target.
if not ALL_FEATURES:
    OHLCV = {"open","high","low","close","volume"}
    ALL_FEATURES = [c for c in df.columns if c not in (OHLCV | {"target"})]

# Wir filtern die Features ggf. noch weiter (z.B. "mom_only" für Momentum-Strategie).
if FEATURES_USED_TAG == "mom_only":
    FEATURES = [c for c in ALL_FEATURES
                if ("logret" in c) or ("macd" in c) or (c in {"sma_diff","rsi_14","bb_pos"})]
else:
    FEATURES = ALL_FEATURES

# Definition von Input (X) und Zielvariable (y)
TARGET = "target"
X = df[FEATURES].copy()
y = df[TARGET].astype(int).copy()
print("FEATURES (final):", FEATURES)

Loaded TRAIN_CSV: ../data/AAPL_1d_2010-01-01_2026-01-01_cls_h1_abs0p0005.csv
[Label] using horizon=1 | mode=abs | epsilon=0.0005
FEATURES (final): ['logret_1d', 'logret_3d', 'logret_5d', 'realized_vol_10', 'bb_pos', 'rsi_14', 'macd', 'macd_sig', 'macd_diff', 'vol_z_20', 'sma_diff']


In [6]:
# === 3) CHRONOLOGISCHER SPLIT (Train/Val/Test) ===
# Finanzdaten dürfen NICHT zufällig gesplittet werden (Zeitabhängigkeit!).
# Wir definieren feste Zeiträume für die Sets.

from sklearn.preprocessing import StandardScaler

# Train: Alles VOR 2024
train_mask = X.index < "2024-01-01"

# Validation: Das Jahr 2024 (zum Tunen der Hyperparameter und Early Stopping)
val_mask   = (X.index >= "2024-01-01") & (X.index < "2025-01-01")

# Test: Das Jahr 2025 (echter "Out-of-Sample" Test)
test_mask  = X.index >= "2025-01-01"

# Daten aufteilen
X_train, y_train = X.loc[train_mask], y.loc[train_mask]
X_val,   y_val   = X.loc[val_mask],   y.loc[val_mask]
X_test,  y_test  = X.loc[test_mask],  y.loc[test_mask]

# Ausgabe der Größen zur Kontrolle
print(f"Split sizes → train {len(X_train)} ({X_train.index.min().date()} - {X_train.index.max().date()})")
print(f"              val   {len(X_val)}   ({X_val.index.min().date()} - {X_val.index.max().date()})")
print(f"              test  {len(X_test)}  ({X_test.index.min().date()} - {X_test.index.max().date()})")

Split sizes → train 3489 (2010-02-22 - 2023-12-29)
              val   252   (2024-01-02 - 2024-12-31)
              test  250  (2025-01-02 - 2025-12-31)


In [7]:
# === 4) SKALIERUNG (StandardScaler) ===
# Neuronale Netze funktionieren am besten, wenn die Inputs klein sind (etwa um -1 bis 1).
# Wir nutzen StandardScaler, um (Wert - Mittelwert) / Standardabweichung zu rechnen.

# WICHTIG: Der Scaler darf NUR auf das Training-Set "gefittet" (berechnet) werden.
# Wir dürfen keine Informationen aus Val/Test (Zukunft) nutzen (Data Leakage!).
scaler = StandardScaler(with_mean=True, with_std=True)

# Fit auf Train, Transform auf Train
X_train_s = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=FEATURES)
# Transform auf Val und Test (mit den Parametern von Train!)
X_val_s   = pd.DataFrame(scaler.transform(X_val),       index=X_val.index,   columns=FEATURES)
X_test_s  = pd.DataFrame(scaler.transform(X_test),      index=X_test.index,  columns=FEATURES)

# Wir speichern den Scaler, damit wir später neue Daten genau gleich skalieren können.
import joblib
joblib.dump(scaler, RUN_DIR / "scaler.joblib")

# Diagnose: Drift-Check
# Ändern sich die statistischen Eigenschaften der Daten zwischen Train und Test stark?
# Das nennt man "Dataset Shift" oder "Concept Drift".
def drift_summary(Xa: pd.DataFrame, Xb: pd.DataFrame):
    out = []
    for c in Xa.columns:
        m1, s1 = Xa[c].mean(), Xa[c].std(ddof=1)
        m2, s2 = Xb[c].mean(), Xb[c].std(ddof=1)
        # Verhältnisse der Standardabweichungen und Differenz der Mittelwerte
        ratio_std = float((s2 + 1e-9) / (s1 + 1e-9))
        diff_mean = float(m2 - m1)
        out.append({"feature": c, "mean_diff": diff_mean, "std_ratio": ratio_std})
    return pd.DataFrame(out).sort_values("std_ratio", ascending=False)

# Ergebnisse in CSV speichern
drift_summary(X_train_s, X_test_s).to_csv(RUN_DIR / "drift_train_vs_test.csv", index=False)

In [8]:
# === 5) WINDOWING (Zeitreihen-Daten vorbereiten) ===
# LSTMs und GRUs benötigen keine Einzelbilder (Vektoren), sondern Sequenzen (Filme).
# Wir müssen die Daten umformen: Aus [Samples, Features] wird [Samples, Timesteps, Features].

def make_windows(X_df: pd.DataFrame, y_ser: pd.Series, lookback: int):
    # Konvertierung zu Numpy Arrays für Speed
    X_values = X_df.values.astype(np.float32)
    y_values = y_ser.values.astype(np.int32)
    n = len(X_df)
    xs, ys = [], []
    # Wir gleiten mit einem Fenster der Größe 'lookback' über die Daten
    for i in range(lookback-1, n):
        # Das Input-Fenster geht von i-lookback+1 bis i (einschließlich)
        xs.append(X_values[i - lookback + 1 : i + 1])
        # Das Label gehört zum Zeitpunkt i
        ys.append(y_values[i])
    return np.stack(xs, axis=0), np.array(ys)

# Anwendung der Window-Funktion auf alle Splits
Xtr_win, ytr = make_windows(X_train_s, y_train, USE_LOOKBACK)
Xva_win, yva = make_windows(X_val_s,   y_val,   USE_LOOKBACK)
Xte_win, yte = make_windows(X_test_s,  y_test,  USE_LOOKBACK)

# Seeds für Zufallsgeneratoren setzen (für Reproduzierbarkeit)
np.random.seed(SEED); tf.random.set_seed(SEED)

# TensorFlow Datasets (tf.data.Dataset) erstellen.
# Diese laden Daten effizient "on demand" und mischen sie im RAM.
def to_ds(X, y, batch, shuffle):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        # Nur Trainingsdaten sollten gemischt werden
        ds = ds.shuffle(buffer_size=len(X), seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(batch).prefetch(tf.data.AUTOTUNE)

# Erstellen der Dataset-Objekte
ds_train = to_ds(Xtr_win, ytr, BATCH, shuffle=ABL_SHUFFLE_TRAIN)
ds_val   = to_ds(Xva_win, yva, BATCH, shuffle=False)
ds_test  = to_ds(Xte_win, yte, BATCH, shuffle=False)

In [9]:
# === 6) DIAGNOSE: LOGISTIC REGRESSION BASELINE ===
# Bevor wir das komplexe LSTM trainieren, testen wir ein simples lineares Modell.
# Wenn unser LSTM schlechter abschneidet als das hier, wissen wir, dass das Problem nicht "fehlende Komplexität" ist.
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

logit = LogisticRegression(max_iter=200)
# Wir trainieren nur auf den features, ohne Zeitreihen-Struktur (Flatten wäre alternativ möglich, hier einfach zeilenweise)
# Um vergleichbar zu bleiben, müssen wir die Indices anpassen (wegen Lookback-Verlust am Anfang)
logit.fit(X_train_s.iloc[USE_LOOKBACK-1:], y_train.iloc[USE_LOOKBACK-1:])

# Vorhersagen auf Validation und Test
y_proba_val_lr = logit.predict_proba(X_val_s.iloc[USE_LOOKBACK-1:])[:,1]
y_proba_test_lr = logit.predict_proba(X_test_s.iloc[USE_LOOKBACK-1:])[:,1]

# Score berechnen (AUROC)
res_val  = roc_auc_score(y_val.iloc[USE_LOOKBACK-1:], y_proba_val_lr)
res_test = roc_auc_score(y_test.iloc[USE_LOOKBACK-1:], y_proba_test_lr)

print(f"[Diag] LogReg AUROC val/test = {res_val:.3f}/{res_test:.3f}")

[Diag] LogReg AUROC val/test = 0.437/0.535


In [10]:
# === 7) MODELLBAU (LSTM/GRU ARCHITEKTUR) ===
# Hier wird das neuronale Netz definiert.

from tensorflow.keras import layers, regularizers, optimizers, callbacks, models

# Wahl der Zelle: GRU ist oft schneller und gleich gut wie LSTM.
rnn_cell = layers.GRU if CELL == "GRU" else layers.LSTM

# Wir bauen das Modell schichtweise auf (Sequential API Liste).
model_layers = [
    # Input Layer: Definiert die Form der Daten (Lookback x Features)
    layers.Input(shape=(USE_LOOKBACK, len(FEATURES))),

    # 1. Rekurrente Schicht (Hidden Layer 1)
    # return_sequences=True gibt die gesamte Sequenz weiter (wichtig für Stacked RNNs)
    rnn_cell(WIDTH1, return_sequences=True, recurrent_dropout=RDROP),
]

# Ablation Layer-Norm: Normalisierung zwischen den Schichten stabilisiert das Training.
if ABL_LN_LAYOUT == "both":
    model_layers.append(layers.LayerNormalization())

# 2. Rekurrente Schicht (Hidden Layer 2)
# return_sequences=False (Standard) gibt nur den letzten Zustand am Ende der Sequenz aus.
model_layers += [
    rnn_cell(WIDTH2, recurrent_dropout=RDROP),
    layers.LayerNormalization(), # Immer nach der letzten RNN Schicht

    # Dense Layer (Fully Connected) zur Merkmals-Verarbeitung
    # L2-Regularisierung bestraft große Gewichte (gegen Overfitting)
    layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(L2_DENSE)),

    # Output Layer: Ein einzelnes Neuron mit Sigmoid-Aktivierung.
    # Gibt eine Wahrscheinlichkeit zwischen 0 und 1 aus.
    layers.Dense(1, activation="sigmoid"),
]

# Modell zusammensetzen
model = models.Sequential(model_layers)

# Kompilieren: Festlegen von Optimizer und Fehlerfunktion (Loss)
model.compile(
    optimizer=optimizers.Adam(learning_rate=LR),
    loss="binary_crossentropy", # Standard-Loss für Ja/Nein Klassifikation
    metrics=[
        tf.keras.metrics.AUC(name="auc"),             # ROC-AUC
        tf.keras.metrics.AUC(name="auprc", curve="PR"), # Precision-Recall AUC (wichtiger bei unbalancierten Daten)
        tf.keras.metrics.BinaryAccuracy(name="acc"),  # Einfache Genauigkeit
        tf.keras.metrics.Precision(name="prec"),      # Wie viele Treffer waren wirklich Treffer?
        tf.keras.metrics.Recall(name="rec"),          # Wie viele aller Treffer haben wir gefunden?
    ],
)

# Callbacks steuern den Trainingsablauf
ckpt_path = RUN_DIR / "best.keras"
cbs = [
    # Checkpoint: Speichert das Modell immer dann, wenn "val_auprc" besser wird.
    callbacks.ModelCheckpoint(filepath=str(ckpt_path),
                              monitor="val_auprc", mode="max",
                              save_best_only=True, verbose=1),
    
    # Early Stopping: Bricht ab, wenn "val_auprc" sich 12 Epochen lang nicht verbessert.
    # restore_best_weights=True stellt sicher, dass wir am Ende das beste Modell haben, nicht das letzte.
    callbacks.EarlyStopping(monitor="val_auprc", mode="max",
                            patience=12, restore_best_weights=True),
    
    # ReduceLROnPlateau: Halbiert die Lernrate, wenn 6 Epochen lang nichts passiert.
    # Das hilft, im Minimum genauer zu konvergieren.
    callbacks.ReduceLROnPlateau(monitor="val_auprc", mode="max",
                                factor=0.5, patience=6, min_lr=1e-5, verbose=1),
]

In [11]:
# === 8) TRAINING STARTEN ===
# Jetzt geht's los. Wir fitten das Modell auf den Trainingsdaten.
# validation_data sorgt dafür, dass wir nach jeder Epoche auf den Val-Daten prüfen.

history = model.fit(ds_train, validation_data=ds_val, epochs=EPOCHS,
                    callbacks=cbs, verbose=1)

# Den Verlauf (History) speichern wir als CSV, um Lernkurven zu plotten.
pd.DataFrame(history.history).to_csv(RUN_DIR / "history.csv", index=False)

Epoch 1/100
54/54 [==============================] - ETA: 0s - loss: 0.7236 - auc: 0.5039 - auprc: 0.5125 - acc: 0.5026 - prec: 0.5124 - rec: 0.5323
Epoch 1: val_auprc improved from -inf to 0.51752, saving model to ..\results\2026-01-03_21-02-35_lstm\best.keras
54/54 [==============================] - 18s 279ms/step - loss: 0.7236 - auc: 0.5039 - auprc: 0.5125 - acc: 0.5026 - prec: 0.5124 - rec: 0.5323 - val_loss: 0.7474 - val_auc: 0.4149 - val_auprc: 0.5175 - val_acc: 0.4301 - val_prec: 0.5195 - val_rec: 0.3540 - lr: 5.0000e-04
Epoch 2/100
54/54 [==============================] - ETA: 0s - loss: 0.6987 - auc: 0.5452 - auprc: 0.5484 - acc: 0.5332 - prec: 0.5407 - rec: 0.5688
Epoch 2: val_auprc improved from 0.51752 to 0.54247, saving model to ..\results\2026-01-03_21-02-35_lstm\best.keras
54/54 [==============================] - 14s 268ms/step - loss: 0.6987 - auc: 0.5452 - auprc: 0.5484 - acc: 0.5332 - prec: 0.5407 - rec: 0.5688 - val_loss: 0.7386 - val_auc: 0.4221 - val_auprc: 0.5425

In [12]:
# === 9) TEST-EVALUATION (Diagnose) ===
# Wir prüfen sofort, wie das Modell auf den ungesehenen Testdaten abschneidet.

test_metrics = model.evaluate(ds_test, return_dict=True, verbose=0)
print("Test (keras) metrics:", json.dumps(test_metrics, indent=2))

# Wir müssen einen Schwellwert (Threshold) wählen, ab dem wir "Wahr" sagen.
# Standard ist 0.5, aber bei Finanzdaten ist oft ein anderer Wert besser.
# Wir optimieren den Threshold auf den Validierungsdaten (maximiere MCC).

val_proba = model.predict(ds_val, verbose=0).ravel()

def choose_threshold(y_true, y_prob, bounds=(0.35, 0.65)):
    # Wir testen alle Wahrscheinlichkeiten als mögliche Thresholds
    uniq = np.unique(y_prob); cand = np.r_[0.0, uniq, 1.0]
    best_t, best_s = 0.5, -1
    for t in cand:
        # Simuliere Prediction mit diesem Threshold
        yp = (y_prob >= t).astype(int)
        # Check ob Positive Rate plausibel ist (innerhalb bounds)
        pr = yp.mean()
        if not (bounds[0] <= pr <= bounds[1]): 
            continue
        # Berechne MCC Score
        s = matthews_corrcoef(y_true, yp)
        if s > best_s: best_s, best_t = s, float(t)
    return best_t, best_s

# Besten Threshold finden
thr_diag, mcc_val_diag = choose_threshold(yva, val_proba, bounds=(0.35, 0.65))
print(f"[Diag] thr@val(max MCC) = {thr_diag:.3f} | val_MCC={mcc_val_diag:.3f}")

# Mit diesem Threshold testen wir nun auf dem Test-Set
y_proba_test = model.predict(ds_test, verbose=0).ravel()
y_pred_diag = (y_proba_test >= thr_diag).astype(int)

# Zusätzliche Metriken berechnen, die Keras nicht standardmäßig ausgibt
extra = {"balanced_accuracy": float(balanced_accuracy_score(yte, y_pred_diag)),
         "mcc": float(matthews_corrcoef(yte, y_pred_diag)),
         "auprc": float(average_precision_score(yte, y_proba_test))}

# Speichern
with open(RUN_DIR / "extra_test_metrics_diag.json", "w") as f:
    json.dump(extra, f, indent=2)

# Confusion Matrix und Klassifikations-Report ausgeben
print("\n[Diag] Confusion (test):\n", confusion_matrix(yte, y_pred_diag))
print("\n[Diag] Report (test):\n", classification_report(yte, y_pred_diag, digits=3))

Test (keras) metrics: {
  "loss": 0.7134947180747986,
  "auc": 0.5023605227470398,
  "auprc": 0.5129700303077698,
  "acc": 0.5026177763938904,
  "prec": 0.5249999761581421,
  "rec": 0.42424243688583374
}
[Diag] thr@val(max MCC) = 0.477 | val_MCC=-0.055

[Diag] Confusion (test):
 [[50 42]
 [53 46]]

[Diag] Report (test):
               precision    recall  f1-score   support

           0      0.485     0.543     0.513        92
           1      0.523     0.465     0.492        99

    accuracy                          0.503       191
   macro avg      0.504     0.504     0.502       191
weighted avg      0.505     0.503     0.502       191



In [13]:
# === 10) SAVE & EXPORT (Artefakte sichern) ===
# Wir speichern alle relevanten Informationen über den Lauf in JSON-Dateien.
# Das ist essenziell für Nachvollziehbarkeit und spätere Auswertung.

env_info = {
    "python": sys.version,
    "tensorflow": tf.__version__,
    "seed": SEED,
    "ticker": TICKER, "start": START, "end": END, "interval": INTERVAL,
    "horizon": HORIZON, "epsilon_mode": EPS_MODE, "epsilon": EPSILON,
    "featureset": FEATURESET, "features_used": FEATURES_USED_TAG,
    "features_final": FEATURES,
    "lookback": USE_LOOKBACK, "batch": BATCH, "epochs": EPOCHS,
    "cell": CELL, "width1": WIDTH1, "width2": WIDTH2,
    "dropout_cfg": DROPOUT, "recurrent_dropout_used": RDROP,
    "ln_layout": ABL_LN_LAYOUT,
    "lr": LR,
    "loss": "BCE",
    "train_csv": TRAIN_CSV,
    "features_yaml": yaml_path,
    "best_config_path": BEST_CFG_PATH,
    "best_checkpoint_path": str(ckpt_path),
}

with open(RUN_DIR / "env_info.json", "w") as f:
    json.dump(env_info, f, indent=2)

# Eine kompakte Config für 'config.json' im Run-Ordner, für schnelle Übersicht
final_cfg_dump = {
    "ticker": TICKER, "start": START, "end": END, "interval": INTERVAL,
    "horizon": HORIZON, "lookback": USE_LOOKBACK,
    "featureset": FEATURESET, "features": FEATURES,
    "scaler": "StandardScaler", "seed": SEED, "batch": BATCH, "epochs": EPOCHS,
    "cell": CELL, "width1": WIDTH1, "width2": WIDTH2,
    "dropout": DROPOUT, "recurrent_dropout_used": RDROP,
    "ln_layout": ABL_LN_LAYOUT,
    "lr": LR,
    "loss": "BCE",
    "epsilon_mode": EPS_MODE, "epsilon": EPSILON,
    "train_csv": TRAIN_CSV,
    "features_yaml": yaml_path,
    "wfcv_best_config_source": BEST_CFG_PATH,
    "ablations": {
        "shuffle_train": ABL_SHUFFLE_TRAIN,
        "no_recurrent_dropout": ABL_NO_RECURRENT_DROPOUT,
        "ln_layout": ABL_LN_LAYOUT,
        "l2_dense": L2_DENSE
    }
}

with open(RUN_DIR / "config.json", "w") as f:
    json.dump(final_cfg_dump, f, indent=2)

# Das trainierte Modell speichern
model.save(RUN_DIR / "model.keras")

# Wahrscheinlichkeiten und Labels speichern (für Backtest-Notebooks)
np.save(RUN_DIR / "y_test.npy", yte)
np.save(RUN_DIR / "y_proba.npy", y_proba_test)

print(f"\nArtefakte gespeichert in: {RUN_DIR}")


Artefakte gespeichert in: ..\results\2026-01-03_21-02-35_lstm
